In [1]:
#import necessary packages
import spotipy
from spotipy import util

from pprint import pprint
import random
import string
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np
import json
import os
import csv

In [2]:
#generate token for spotify API
username = 'ernyjyqoecerasugt8dpmhbmo'
scope = 'playlist-modify-public'
my_client_id = '83c842b2f4d447b6b2a19df91847487d'
my_client_secret = '8bd6101237114d9eb851e2d3183affce'

spotify_token = util.prompt_for_user_token(username, 
                                           scope, 
                                           client_id=my_client_id, 
                                           client_secret=my_client_secret,
                                           redirect_uri='http://example.com/callback/')

sp = spotipy.Spotify(auth=spotify_token)

In [3]:
#import Billboard CSV

#ChartsDF = pd.read_csv("Resources/Hot_100BBCharts_time.csv")
#ChartsDF = pd.read_csv("Resources/test_Hot_100BBCharts_time.csv")
#ChartsDF = pd.read_csv("Resources/Head_test_Hot_100BBCharts_time.csv")

#ChartsDF = pd.read_csv("Resources/HotList_Splits_Dir/xaa.csv")
#ChartsDF = pd.read_csv("Resources/HotList_Splits_Dir/xab.csv")
ChartsDF = pd.read_csv("Resources/HotList_Splits_Dir/xac.csv")
#ChartsDF = pd.read_csv("Resources/HotList_Splits_Dir/xad.csv")
#ChartsDF = pd.read_csv("Resources/HotList_Splits_Dir/xae.csv")
#ChartsDF = pd.read_csv("Resources/HotList_Splits_Dir/xaf.csv")
#ChartsDF = pd.read_csv("Resources/HotList_Splits_Dir/xag.csv")
#ChartsDF = pd.read_csv("Resources/HotList_Splits_Dir/xah.csv")
#ChartsDF = pd.read_csv("Resources/HotList_Splits_Dir/xai.csv")
#ChartsDF = pd.read_csv("Resources/HotList_Splits_Dir/xaj.csv")

#ChartsDF = pd.read_csv("Resources/TEST_Billboard_1960s.csv", encoding = "ISO-8859-1")
#ChartsDF = pd.read_csv("Resources/Billboard_1960s.csv", encoding = "ISO-8859-1")
#ChartsDF = pd.read_csv("Resources/Billboard_1970s.csv", encoding = "ISO-8859-1")
#ChartsDF = pd.read_csv("Resources/Billboard_1980s.csv", encoding = "ISO-8859-1")
#ChartsDF = pd.read_csv("Resources/Billboard_1990s.csv", encoding = "ISO-8859-1")
#ChartsDF = pd.read_csv("Resources/Billboard_2000s.csv", encoding = "ISO-8859-1")
#ChartsDF = pd.read_csv("Resources/Billboard_2010s.csv", encoding = "ISO-8859-1")
#ChartsDF.head()

In [4]:
#Only keep song Title, Artist, and peak chart position

ChartsDF_small = ChartsDF[["Title","Artist","Peak_Position"]]
ChartsDF_small["Song, Artist"] = ChartsDF_small["Title"]+", "+ChartsDF_small["Artist"]
#ChartsDF_small.head()

/Users/william_nashs_mac/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
#Group DF by Song, Artist
GroupedChartsDF = ChartsDF_small.groupby(['Song, Artist']).min()
#GroupedChartsDF.head()

In [6]:
#Create column to store results of fuzzy match

dfcolumnsList = ["Song ID","Song Popularity","Acousticness","Danceability","Energy","Liveness","Speechiness","Valence","Tempo",
                "Instrumentalness","Loudness"]
GroupedChartsDF = GroupedChartsDF.reindex(columns=[*GroupedChartsDF.columns.tolist(), *dfcolumnsList])

#GroupedChartsDF

In [ ]:
#Loop through songs and find best match from Spotify API
song_ids = []
songPopularity = []
acousticness = []
danceability = []
energy = []
liveness = []
speechiness = []
valence = []
tempo = []
instrumentalness = []
loudness =[]

attempts = 3000
songcounter = 0

for index, row in GroupedChartsDF.iterrows():
    
    #index contains: song tile and artist "Africa, Weezer"
    
    songcounter += 1
    print(f"\n Fetching for: {songcounter}:: {index}\n")
    
    if attempts == 3000:
        spotify_token = util.prompt_for_user_token(username, 
                                           scope, 
                                           client_id=my_client_id, 
                                           client_secret=my_client_secret,
                                           redirect_uri='http://example.com/callback/')

        sp = spotipy.Spotify(auth=spotify_token)
        
        attempts = 0
        
        print("token reset")
    
    attempts += 1
    
    test_song = index

    results = sp.search(q=test_song, type='track', limit=10)

    search_match_score = []
    
    if results is None:
        
        song_ids.append("")
        songPopularity.append("")
        danceability.append("")
        energy.append("")
        loudness.append("")
        speechiness.append("")
        acousticness.append("")
        instrumentalness.append("")
        liveness.append("")
        valence.append("")
        tempo.append("") 
        
    else:
        
        if results["tracks"]["total"] > 0:
            
            for x in range(len(results["tracks"]["items"])):
                song = results["tracks"]["items"][x]["name"]
                artist = results["tracks"]["items"][x]["artists"][0]["name"]
                song_id = results["tracks"]["items"][x]["id"]
                song_popularity = results["tracks"]["items"][x]["popularity"]            
                search_match_score.append(fuzz.ratio(test_song,song+","+artist))

            max_loc = 0
            max_val = 0
            
            for y in range(len(results["tracks"]["items"])):
                
                if max_val < search_match_score[y]:
                    max_loc = y
                    max_val = search_match_score[y]
        
            if max_val >= 75:
            
                #print(f"SongID after max_val: {song_id}\n")
                song_ids.append(song_id)
                songPopularity.append(song_popularity)
              
                features = sp.audio_features(song_id)
                
                xstuff = features
            
                if str(xstuff) == "[None]":    
            
                    danceability.append("")
                    energy.append("")
                    loudness.append("")
                    speechiness.append("")
                    acousticness.append("")
                    instrumentalness.append("")
                    liveness.append("")
                    valence.append("")
                    tempo.append("")
            
                else:  
                    for feature in features:
                        #print(f"songID: {song_id}\n")
                        #print(json.dumps(feature, indent=4))
                    
                        danceability.append(feature["danceability"])
                        energy.append(feature["energy"])
                        loudness.append(feature["loudness"])
                        speechiness.append(feature["speechiness"])
                        acousticness.append(feature["acousticness"])
                        instrumentalness.append(feature["instrumentalness"])
                        liveness.append(feature["liveness"])
                        valence.append(feature["valence"])
                        tempo.append(feature["tempo"]) 
            else:
                song_ids.append("")
                songPopularity.append("")
                danceability.append("")
                energy.append("")
                loudness.append("")
                speechiness.append("")
                acousticness.append("")
                instrumentalness.append("")
                liveness.append("")
                valence.append("")
                tempo.append("") 
        
        else:
            song_ids.append("")
            songPopularity.append("")
            danceability.append("")
            energy.append("")
            loudness.append("")
            speechiness.append("")
            acousticness.append("")
            instrumentalness.append("")
            liveness.append("")
            valence.append("")
            tempo.append("") 
        
        
GroupedChartsDF["Song ID"] = song_ids
GroupedChartsDF["Song Popularity"] = songPopularity
GroupedChartsDF["Acousticness"] = acousticness
GroupedChartsDF["Danceability"] = danceability
GroupedChartsDF["Energy"] = energy
GroupedChartsDF["Liveness"] = liveness
GroupedChartsDF["Speechiness"] = speechiness
GroupedChartsDF["Valence"] = valence
GroupedChartsDF["Tempo"] = tempo
GroupedChartsDF["Instrumentalness"] = instrumentalness
GroupedChartsDF["Loudness"] = loudness

#GroupedChartsDF.to_csv('Resources/Features_Hot_100BBCharts_time.csv', index=False)
#GroupedChartsDF.to_csv('Resources/Features_Head_test_Hot_100BBCharts_time.csv', index=False)

#GroupedChartsDF.to_csv("Resources/Feature_xaa.csv", index=False)
#GroupedChartsDF.to_csv("Resources/Feature_xab.csv", index=False)
GroupedChartsDF.to_csv("Resources/Feature_xac.csv", index=False)
#GroupedChartsDF.to_csv("Resources/Feature_xad.csv", index=False)
#GroupedChartsDF.to_csv("Resources/Feature_xae.csv", index=False)
#GroupedChartsDF.to_csv("Resources/Feature_xaf.csv", index=False)
#GroupedChartsDF.to_csv("Resources/Feature_xag.csv", index=False)
#GroupedChartsDF.to_csv("Resources/Feature_xah.csv", index=False)
#GroupedChartsDF.to_csv("Resources/Feature_xai.csv", index=False)
#GroupedChartsDF.to_csv("Resources/Feature_xaj.csv", index=False)

#GroupedChartsDF.to_csv('Resources/SongIDs_Billboard_1960s.csv', index=False)
#GroupedChartsDF.to_csv('Resources/SongIDs_Billboard_1970s.csv', index=False)
#GroupedChartsDF.to_csv('Resources/SongIDs_Billboard_1980s.csv', index=False)
#GroupedChartsDF.to_csv('Resources/SongIDs_Billboard_1990s.csv', index=False)
#GroupedChartsDF.to_csv('Resources/SongIDs_Billboard_2000s.csv', index=False)
#GroupedChartsDF.to_csv('Resources/SongIDs_Billboard_2010s.csv', index=False)

print(f"That's all... done\n")
#GroupedChartsDF


 Fetching for: 1:: "Cherry Cherry" from Hot August Night, Neil Diamond

token reset

 Fetching for: 2:: "Having A Party" Medley, The Ovations (Featuring Louis Williams)


 Fetching for: 3:: "Joy" Pt. I, Isaac Hayes


 Fetching for: 4:: "Roots" Medley, Quincy Jones


 Fetching for: 5:: #9 Dream, John Lennon


 Fetching for: 6:: 'til I Can Make It On My Own, Tammy Wynette


 Fetching for: 7:: 'til It's Time To Say Goodbye, Jonathan Cain


 Fetching for: 8:: (All I Have To Do Is) Dream, The Nitty Gritty Dirt Band


 Fetching for: 9:: (Call Me Your) Anything Man, Bobby Moore


 Fetching for: 10:: (Do The) Push And Pull Part I, Rufus Thomas


 Fetching for: 11:: (Don't Fear) The Reaper, Blue Oyster Cult


 Fetching for: 12:: (Don't Worry) If There's A Hell Below We're All Going To Go, Curtis Mayfield


 Fetching for: 13:: (For God's Sake) Give More Power To The People, The Chi-lites


 Fetching for: 14:: (Hey Won't You Play) Another Somebody Done Somebody Wrong Song, B.J. Thomas


 Fetchin


 Fetching for: 127:: All My Trials, Ray Stevens


 Fetching for: 128:: All Right Now, Lea Roberts


 Fetching for: 129:: All Roads (Lead Back To You), Donny Most


 Fetching for: 130:: All Shook Up, Suzi Quatro


 Fetching for: 131:: All Strung Out On You, John Travolta


 Fetching for: 132:: All The Kings Horses, Aretha Franklin


 Fetching for: 133:: All The Young Dudes, Mott The Hoople


 Fetching for: 134:: Almost Saturday Night, John Fogerty


 Fetching for: 135:: Alone Again (Naturally), Gilbert O'Sullivan


 Fetching for: 136:: Already Gone, Eagles


 Fetching for: 137:: Also Sprach Zarathustra (2001), Deodato


 Fetching for: 138:: Alvin Stone (The Birth & Death Of A Gangster), Fantastic Four


 Fetching for: 139:: Always, Luther Ingram


 Fetching for: 140:: Am I Black Enough For You, Billy Paul


 Fetching for: 141:: Am I Losing You, The Partridge Family Starring Shirley Jones Featuring David Cassidy


 Fetching for: 142:: Amanda, Dionne Warwicke


 Fetching for: 143:: Amazi


 Fetching for: 268:: Beautiful, Gordon Lightfoot


 Fetching for: 269:: Bed And Board, Barbara Mason


 Fetching for: 270:: Bed Of Rose's, The Statler Brothers


 Fetching for: 271:: Bedtime Story, Tammy Wynette


 Fetching for: 272:: Been To Canaan, Carole King


 Fetching for: 273:: Been Too Long On The Road, Mark Lindsay


 Fetching for: 274:: Beer Barrel Polka/Dick And Jane, Bobby Vinton


 Fetching for: 275:: Before The Next Teardrop Falls, Freddy Fender


 Fetching for: 276:: Beg, Steal Or Borrow, The New Seekers


 Fetching for: 277:: Beginning To Feel The Pain, Mac Davis


 Fetching for: 278:: Beginnings/Colour My World, Chicago


 Fetching for: 279:: Behind Blue Eyes, The Who


 Fetching for: 280:: Behind Closed Doors, Charlie Rich


 Fetching for: 281:: Believe Half Of What You See (And None Of What You Hear), Leon Haywood


 Fetching for: 282:: Bell Bottom Blues, Derek & The Dominos


 Fetching for: 283:: Bell Bottom Blues, Eric Clapton


 Fetching for: 284:: Ben, Michael J